In [133]:
# 다운로드가 필요한 모듈, 라이브러리
# pip install lightgbm
# pip install catboost
# pip install soccerdata

In [134]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, log_loss

from soccerdata.fbref import FBref 
from pathlib import Path

# 학습 모델들
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier


In [135]:
# 1) CSV 불러오기 & match_id 생성
data = pd.read_csv('Matches.csv', parse_dates=['MatchDate'])
data = data.reset_index().rename(columns={'index':'match_id'})

# 2) 홈/원정 각각 long 포맷으로 전환
home = data[['match_id','MatchDate','HomeTeam','FTHome','FTAway']].copy()
home = home.assign(
    team           = home['HomeTeam'],
    goals_for      = home['FTHome'],
    goals_against  = home['FTAway'],
    venue          = 'Home'
)[['match_id','MatchDate','team','goals_for','goals_against','venue']]

away = data[['match_id','MatchDate','AwayTeam','FTAway','FTHome']].copy()
away = away.assign(
    team           = away['AwayTeam'],
    goals_for      = away['FTAway'],
    goals_against  = away['FTHome'],
    venue          = 'Away'
)[['match_id','MatchDate','team','goals_for','goals_against','venue']]

matches_long = pd.concat([home, away], ignore_index=True)

# 3) 정렬하고 인덱스 재설정 (꼭 필요)
matches_long = matches_long.sort_values(['team','MatchDate']).reset_index(drop=True)

# 4) 과거 3·5경기 득실 합계 계산 (transform 이용)
for N in (3, 5):
    # 먼저 “현재 경기” 제외를 위해 shift()
    shifted_gf = matches_long.groupby('team')['goals_for']     .shift()
    shifted_ga = matches_long.groupby('team')['goals_against'] .shift()

    # rolling 합계 계산
    matches_long[f'GF{N}'] = (shifted_gf
                              .groupby(matches_long['team'])
                              .transform(lambda x: x.rolling(N).sum()))
    matches_long[f'GA{N}'] = (shifted_ga
                              .groupby(matches_long['team'])
                              .transform(lambda x: x.rolling(N).sum()))

# 5) 홈/Away별로 다시 뽑아서 이름 바꾸기
home_stats = (
    matches_long[matches_long['venue']=='Home']
    .set_index('match_id')[['GF3','GA3','GF5','GA5']]
    .rename(columns={
        'GF3':'GF3Home','GA3':'GA3Home',
        'GF5':'GF5Home','GA5':'GA5Home'
    })
)
away_stats = (
    matches_long[matches_long['venue']=='Away']
    .set_index('match_id')[['GF3','GA3','GF5','GA5']]
    .rename(columns={
        'GF3':'GF3Away','GA3':'GA3Away',
        'GF5':'GF5Away','GA5':'GA5Away'
    })
)

# 6) map으로 원본 data에 컬럼 추가
for col in home_stats.columns:
    data[col] = data['match_id'].map(home_stats[col])
for col in away_stats.columns:
    data[col] = data['match_id'].map(away_stats[col])

# 7) 불필요해진 match_id 제거 (선택)
data = data.drop(columns=['match_id'])

# 8) 결과 확인
# data.info()

In [136]:
data = data[data['Division'] == 'E0']   # 프리미어 리그(epl) 데이터 추출

# 1-1 xg 데이터 불러오기
xg_data = pd.read_csv('xg_data.csv')

# 1-2 xg 데이터와 Matches 데이터 합치기

# 날짜 칼럼을 datetime.date 로 맞추기
xg_data['MatchDate'] = pd.to_datetime(xg_data['MatchDate']).dt.date
data['MatchDate'] = pd.to_datetime(data['MatchDate']).dt.date 

# Mathces와 다른 팀명들을 모두 동일하도록 mapping
team_name_map = {
    'Manchester City'   : 'Man City',
    'Manchester United' : 'Man United',
    'Newcastle United' : 'Newcastle',
    'Norttingham Forest' : 'Nottm Forest',
    'Wolverhampton Wanderers' : 'Wolves',
    'West Bromwich Albion' : 'West Brom',
}   

# xg_data 에 적용 (Home / Away 양쪽)
xg_data['HomeTeam'] = xg_data['HomeTeam'].replace(team_name_map)
xg_data['AwayTeam'] = xg_data['AwayTeam'].replace(team_name_map)



# MatchDate를 시계열 데이터로 전환
xg_data['MatchDate'] = pd.to_datetime(xg_data['MatchDate'])  # MatchDate를 datetime 형식으로 변환
data['MatchDate'] = pd.to_datetime(data['MatchDate'])  # MatchDate를 datetime 형식으로 변환
data = data[(data['MatchDate'] > '2016-08-13')]


# xg_data와 Matches 데이터 합치기
data_final = data.merge(
    xg_data,
    on=['MatchDate', 'HomeTeam', 'AwayTeam'],
    how='inner',          # 이전에 left
    validate='1:1'        # 같은 키가 중복되면 오류로 알려줌
)

In [137]:
# 2. 데이터 전처리
# 2-1 데이터 전처리 :데이터 연도 기준 필터링
# 프리미어 리그(epl)의 2022~2023연도 데이터를 사용


# 2-2 데이터 전처리 : 학습에 사용할 column만 추출
# 'MatchTime' 제거
columns = ['MatchDate', 'HomeTeam', 'AwayTeam', 'HomeElo', 'AwayElo', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'OddHome', 'OddDraw', 'OddAway', 'FTResult', 'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 'HandiSize', 'HandiHome', 'HandiAway', 'GF3Home', 'GA3Home', 'GF5Home', 'GA5Home', 'GF3Away', 'GA3Away', 'GF5Away', 'GA5Away', 'home_goals_l3', 'home_goals_l5', 'away_goals_l3', 'away_goals_l5', 'home_xg_l3', 'home_xg_l5', 'away_xg_l3', 'away_xg_l5'
]
data_final = data_final[columns]

In [ ]:
# 2-3 데이터 전처리 : 결측치 확인
data_final.isnull().sum()

# 결측치는 학습에 영향을 주지 않도록 모두 0으로 처리 
columns_NaN = ['home_goals_l3', 'home_goals_l5', 'away_goals_l3', 'away_goals_l5', 'home_xg_l3', 'home_xg_l5', 'away_xg_l3', 'away_xg_l5']

# 결측치 있는 행 제거
data_final[columns_NaN] = data_final[columns_NaN].fillna(0)

# data_final = data_final.dropna(subset=columns).copy()

In [139]:
# 2-4 데이터 전처리 : Standardize(표준화), OneHotEncoding(원-핫 인코딩)
log_columns = ['HomeElo', 'AwayElo', 'OddHome', 'OddDraw', 'OddAway', 'MaxHome', 'MaxDraw', 'MaxAway', 'Over25', 'Under25', 'MaxOver25', 'MaxUnder25', 'HandiHome', 'HandiAway'] # 로그 변환할 columns
standarize_columns = ['HomeElo', 'AwayElo', 'OddHome', 'OddDraw', 'OddAway', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'HandiSize', 'HandiHome', 'HandiAway', 'GF3Home', 'GA3Home', 'GF5Home', 'GA5Home', 'GF3Away', 'GA3Away', 'GF5Away', 'GA5Away']  # 표준화할 columns
encoding_columns = ['HomeTeam', 'AwayTeam']  # 원-핫 인코딩할 columns

# 분산이 큰 배당률 관련 columns와 Elo columns는 표준화 전에 log scale을 먼저 적용
data_final[log_columns] = np.log1p(data_final[log_columns])

# Pipeline에서 전처리 해줄 ColumnsTransformer 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('standardize', StandardScaler(), standarize_columns),  # 표준화
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=True), encoding_columns)  # 원-핫 인코딩
    ],
    remainder='passthrough'  # 나머지 컬럼은 그대로 유지
)


In [ ]:
# 팀의 전력 차이를 예측하기 위한 파생 column 생성

# home과 away의 xG차이
data['xg_l3_diff'] = data['home_xg_l3'] - data['away_xg_l3']  
data['xg_l5_diff'] = data['home_xg_l5'] - data['away_xg_l5']

# home과 away의 Elo 차이
data['elo_diff'] = data['HomeElo'] - data['AwayElo']  

data[]




In [141]:
data_train = data_final[(data_final['MatchDate'] < '2025-01-01') & (data_final['MatchDate'] > '2016-08-13')]   # 학습 데이터 : 24/25 시즌제외 모든 데이터 추출
data_test = data_final[(data_final['MatchDate'] >= '2025-01-01')]   # 테스트 데이터 : 24/25 시즌 데이터 추출

# MatchDate 제거
data_train = data_train.drop(columns=['MatchDate']) 
data_test = data_test.drop(columns=['MatchDate'])  

#  featuer, target 분리

X_train = data_train.drop(columns=['FTResult'])  # Feature columns
X_test = data_test.drop(columns=['FTResult'])  # Feature columns
y_train = data_train['FTResult'].map({'H':0, 'D':1, 'A':2})  # Target column
y_test = data_test['FTResult'].map({'H':0, 'D':1, 'A':2})  # Target column

In [142]:
# 학습, 테스트 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y,
#     test_size=0.2,           # 20 % 검증(또는 0.25 등)
#     random_state=42,         
#     stratify=y               # 클래스 비율 유지 
# )

In [143]:
# lighstgbm 파이프 라인 정의

weights = {0 : 1.0, 1 : 12, 2 : 1.0}  # 클래스 가중치 : 무승부에만 12배

pipe_lightgbm = Pipeline(
    steps=[
        ('preprocessor', preprocessor),  # 전처리 단계
        ('classifier', LGBMClassifier(
            objective='multiclass',  # 다중 클래스 분류
            num_class=3,  # 홈 승, 무승부, 원정 승
            n_estimators   = 1400,
            learning_rate  = 0.035,
            max_depth      = -1,          # 자동
            num_leaves     = 63,          # 2^(max_depth) -1 근사
            min_data_in_leaf = 25,        # 최소 leaf 노드 데이터 수
            colsample_bytree = 0.8,
            subsample        = 0.8,
            reg_alpha        = 0.1,
            reg_lambda       = 1.0,
            random_state     = 42,
            class_weight     = weights,
        ))  # LightGBM 
    ]
)

# catboost 파이프 라인 정의
pipe_catboost = Pipeline(
    steps = [
        ('preprocessor', preprocessor),  # 전처리 단계
        ('classifier', CatBoostClassifier(
            loss_function='MultiClass', # 다중 클래스
            iterations = 1200,          # 트리 개수
            learning_rate = 0.05,       # 학습률
            depth = 6,                  # 트리 깊이
            l2_leaf_reg = 3,            # L2 정규화
            random_seed = 42,           
        ))  # CatBoost
    ]
)


pipe_logistic = Pipeline(
    steps=[
        ('preprocessor', preprocessor),  # 전처리 단계
        ('classifier', LogisticRegression(
            max_iter = 1000,              # 최대 반복 횟수
            multi_class = 'multinomial',  # 다중 클래스 분류
            class_weight= 'balanced',     # 클래스 불균형 처리
            solver= 'lbfgs',              # 최적화 알고리즘
        ))  # 로지스틱 회귀
    ]

)

In [144]:
# stacking 모델 정의
stack = StackingClassifier(
    estimators = [
        ('lightgbm', pipe_lightgbm),
        ('catboost', pipe_catboost),
        ('logistic', pipe_logistic)
    ],
    final_estimator= LogisticRegression(max_iter=1000, multi_class='multinomial'),
    stack_method = 'predict_proba',  # 확률 예측을 위한 stacking
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42),  # 교차 검증 설정
    n_jobs= -1  # 모든 CPU 코어 사용
)

In [ ]:
# 1) 홈팀 rolling reature
home_cols = [c for c in feat_df.columns if c.startswith('home_')]
home_feat = (feat_df[['game_id','MatchDate','HomeTeam', *home_cols]]
             .drop_duplicates())

stats = stats.merge(
    home_feat,
    on=['game_id','MatchDate','HomeTeam'],
    how='left'
)

# 2) 원정팀 rolling reature
away_cols = [c for c in feat_df.columns if c.startswith('away_')]
away_feat = (feat_df[['game_id','MatchDate','AwayTeam', *away_cols]]
             .drop_duplicates())

stats = stats.merge(
    away_feat,
    on=['game_id','MatchDate','AwayTeam'],
    how='left'
)

In [145]:
# 모델 학습
# pipe_lightgbm.fit(X_train, y_train)
stack.fit(X_train, y_train)

[07/03/25 13:09:43] WARNING  c:\project\module_project\.venv\lib\site-packages\sklearn\linear_model ]8;id=798906;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=860217;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             \_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in                    
                             version 1.5 and will be removed in 1.7. From then on, it will always                  
                             use 'multinomial'. Leave it to its default value to avoid this                        
                             warning.                                                                              
                               warnings.warn(                                                                      
                                                                                                                   

StackingClassifier(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimators=[('lightgbm',
                                Pipeline(steps=[('preprocessor',
                                                 ColumnTransformer(remainder='passthrough',
                                                                   transformers=[('standardize',
                                                                                  StandardScaler(),
                                                                                  ['HomeElo',
                                                                                   'AwayElo',
                                                                                   'OddHome',
                                                                                   'OddDraw',
                                                                                   'OddAway',
                                                                                   'Form3Home',
                                                                                   'Form5Home',
                                                                                   'Form3Away',
                                                                                   'Form5Away',
                                                                                   'HandiSize',
                                                                                   'HandiHome',
                                                                                   'Han...
                                                                                   'GF5Home',
                                                                                   'GA5Home',
                                                                                   'GF3Away',
                                                                                   'GA3Away',
                                                                                   'GF5Away',
                                                                                   'GA5Away']),
                                                                                 ('onehot',
                                                                                  OneHotEncoder(handle_unknown='ignore'),
                                                                                  ['HomeTeam',
                                                                                   'AwayTeam'])])),
                                                ('classifier',
                                                 LogisticRegression(class_weight='balanced',
                                                                    max_iter=1000,
                                                                    multi_class='multinomial'))]))],
                   final_estimator=LogisticRegression(max_iter=1000,
                                                      multi_class='multinomial'),
                   n_jobs=-1, stack_method='predict_proba')

In [ ]:
# 예측
# y_pred_lightgbm  = pipe_lightgbm.predict(X_test)
# y_prob_lightgbm  = pipe_lightgbm.predict_proba(X_test)   # shape = (n_samples, 3)

y_pred = stack.predict(X_test)
y_prob = stack.predict_proba(X_test)

[07/03/25 13:09:44] WARNING  c:\project\module_project\.venv\lib\site-packages\sklearn\utils\valida ]8;id=87906;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=235884;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             tion.py:2739: UserWarning: X does not have valid feature names, but                   
                             LGBMClassifier was fitted with feature names                                          
                               warnings.warn(                                                                      
                                                                                                                   

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25


[07/03/25 13:09:45] WARNING  c:\project\module_project\.venv\lib\site-packages\sklearn\utils\valida ]8;id=117283;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=30296;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             tion.py:2739: UserWarning: X does not have valid feature names, but                   
                             LGBMClassifier was fitted with feature names                                          
                               warnings.warn(                                                                      
                                                                                                                   

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25


In [147]:
# 무승부 후처리
delta = np.abs(y_prob[:,0] - y_prob[:,2])
mask  = (y_prob[:,0] > .4) & (y_prob[:,2] > .4) & (delta < .05)
y_pred_force = y_pred.copy()
y_pred_force[mask] = 1             # mask 범위에 해당하는 값들은 무승부로 배정

In [148]:

print("Accuracy :", accuracy_score(y_test, y_pred_force))
print("\nClassification Report\n", classification_report(y_test, y_pred_force))

# (선택) 로그-로스 — 다중 클래스 확률 평가
print("Log-loss :", log_loss(y_test, y_prob))

# (선택) 혼동 행렬
print("Confusion Matrix\n", confusion_matrix(y_test, y_pred_force))

Accuracy : 0.5780346820809249


                    WARNING  c:\project\module_project\.venv\lib\site-packages\sklearn\metrics\_cla ]8;id=618153;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=362902;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             ssification.py:1565: UndefinedMetricWarning: Precision is ill-defined                 
                             and being set to 0.0 in labels with no predicted samples. Use                         
                             `zero_division` parameter to control this behavior.                                   
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

                    WARNING  c:\project\module_project\.venv\lib\site-packages\sklearn\metrics\_cla ]8;id=71316;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=729214;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             ssification.py:1565: UndefinedMetricWarning: Precision is ill-defined                 
                             and being set to 0.0 in labels with no predicted samples. Use                         
                             `zero_division` parameter to control this behavior.                                   
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   

                    WARNING  c:\project\module_project\.venv\lib\site-packages\sklearn\metrics\_cla ]8;id=634785;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=768143;file://C:\Users\User\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             ssification.py:1565: UndefinedMetricWarning: Precision is ill-defined                 
                             and being set to 0.0 in labels with no predicted samples. Use                         
                             `zero_division` parameter to control this behavior.                                   
                               _warn_prf(average, modifier, f"{metric.capitalize()} is",                           
                             len(result))                                                                          
                                                                                                                   


Classification Report
               precision    recall  f1-score   support

           0       0.54      0.92      0.68        71
           1       0.00      0.00      0.00        37
           2       0.67      0.54      0.60        65

    accuracy                           0.58       173
   macro avg       0.40      0.48      0.43       173
weighted avg       0.47      0.58      0.50       173

Log-loss : 0.9377171393585088
Confusion Matrix
 [[65  0  6]
 [26  0 11]
 [30  0 35]]
